# Hybrid Model using game tags

First only content based filtering

In [1]:
#get the data 
csvPath = "../Datasets/dataWithRatings.csv"

In [9]:
import pandas as pd
df = pd.read_csv (csvPath)
df

,user_id,name,hours,desc_snippet,all_reviews,release_date,developer,publisher,popular_tags,game_details,...,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price,Rating M1,Rating M2
0,151603712,Fallout 4,87,"Bethesda Game Studios, the award-winning creat...","Mostly Positive,(90,387),- 70% of the 90,387 u...","Nov 9, 2015",Bethesda Game Studios,"Bethesda Softworks,Bethesda Softworks","Open World,Post-apocalyptic,Exploration,Single...","Single-player,Steam Achievements,Full controll...",...,84.0,RPG,"About This Game Bethesda Game Studios, the aw...",NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,$14.99,5,5
1,87445402,Fallout 4,83,"Bethesda Game Studios, the award-winning creat...","Mostly Positive,(90,387),- 70% of the 90,387 u...","Nov 9, 2015",Bethesda Game Studios,"Bethesda Softworks,Bethesda Softworks","Open World,Post-apocalyptic,Exploration,Single...","Single-player,Steam Achievements,Full controll...",...,84.0,RPG,"About This Game Bethesda Game Studios, the aw...",NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,$14.99,5,5
2,25096601,Fallout 4,"1,6","Bethesda Game Studios, the award-winning creat...","Mostly Positive,(90,387),- 70% of the 90,387 u...","Nov 9, 2015",Bethesda Game Studios,"Bethesda Softworks,Bethesda Softworks","Open World,Post-apocalyptic,Exploration,Single...","Single-player,Steam Achievements,Full controll...",...,84.0,RPG,"About This Game Bethesda Game Studios, the aw...",NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,$14.99,1,1
3,211925330,Fallout 4,133,"Bethesda Game Studios, the award-winning creat...","Mostly Positive,(90,387),- 70% of the 90,387 u...","Nov 9, 2015",Bethesda Game Studios,"Bethesda Softworks,Bethesda Softworks","Open World,Post-apocalyptic,Exploration,Single...","Single-player,Steam Achievements,Full controll...",...,84.0,RPG,"About This Game Bethesda Game Studios, the aw...",NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,$14.99,3,5
4,115396529,Fallout 4,"17,9","Bethesda Game Studios, the award-winning creat...","Mostly Positive,(90,387),- 70% of the 90,387 u...","Nov 9, 2015",Bethesda Game Studios,"Bethesda Softworks,Bethesda Softworks","Open World,Post-apocalyptic,Exploration,Single...","Single-player,Steam Achievements,Full controll...",...,84.0,RPG,"About This Game Bethesda Game Studios, the aw...",NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,$14.99,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36252,154230723,Khet 2.0,"1,9",Officially licensed version of the “Mensa Sele...,"Very Positive,(79),- 92% of the 79 user review...","Oct 1, 2014",BlueLine Games,"BlueLine Games,BlueLine Games","Strategy,Casual,Indie,Board Game,Singleplayer,...","Single-player,Multi-player,Cross-Platform Mult...",...,54.0,"Casual,Indie,Strategy",About This Game Officially licensed version o...,NaN,"Minimum:,OS:,Windows XP or better,Processor:,1...","Recommended:,OS:,Windows XP or better,Processo...",$9.99,NaN,1,5
36253,116564064,SAMURAI WARRIORS 4-II,"2,2","The latest iteration of the 10th anniversary ""...","Mostly Positive,(860),- 77% of the 860 user re...","Sep 29, 2015","KOEI TECMO GAMES CO., LTD.","KOEI TECMO GAMES CO., LTD.,KOEI TECMO GAMES CO...","Action,Hack and Slash,Historical,Character Cus...","Single-player,Multi-player,Co-op,Steam Achieve...",...,54.0,Action,About This Game The last title released for t...,NaN,"Minimum:,OS:,Windows® Vista, Windows® 7, Windo...","Recommended:,OS:,Windows® Vista, Windows® 7, W...",$49.99,NaN,1,5
36254,135400225,Life is Hard,"3,1",Life is Hard is a godsim town simulator game d...,"Mixed,(252),- 64% of the 252 us

In [87]:
#select a random user that has played more that minHours #JUST LIKE LAB1
minHours = 500
df['hours'] = df['hours'].str.replace(',', '.')
df['hours'] = df['hours'].astype(float)
users_ratings = df.groupby('user_id').hours.agg(['sum']).reindex(df.user_id).reset_index()
# users_ratings = df.groupby('user_id').total.agg(['sum']).reindex(df.idx).reset_index()
display(users_ratings)
selected = users_ratings['sum'] > minHours

selected_users = df.loc[selected]

random_selected = selected_users.sample() 
select_column_df = random_selected.reset_index()['user_id'] 
selected_user = select_column_df.iloc[0] 
print("Selected user: " + str(selected_user))

,user_id,sum
0,151603712,134.5
1,87445402,87.7
2,25096601,208.0
3,211925330,848.0
4,115396529,365.7
...,...,...
36252,154230723,923.1
36253,116564064,489.0
36254,135400225,1205.5
36255,135400225,1205.5


Selected user: 13190476


In [72]:
def tokenization(text):
    return text.split(",")

In [144]:
df1 = df.drop_duplicates(subset = ["name"])
df1 = df1[["name","popular_tags"]].reset_index(drop=True)
df1 = df1[df1["popular_tags"].notna()].reset_index(drop=True)
df1

,name,popular_tags
0,Fallout 4,"Open World,Post-apocalyptic,Exploration,Single..."
1,Left 4 Dead 2,"Zombies,Co-op,FPS,Multiplayer,Shooter,Action,O..."
2,HuniePop,"Nudity,Mature,Anime,Sexual Content,Dating Sim,..."
3,Path of Exile,"Free to Play,Action RPG,Hack and Slash,RPG,Mul..."
4,Poly Bridge,"Building,Simulation,Physics,Puzzle,Singleplaye..."
...,...,...
1508,Khet 2.0,"Strategy,Casual,Indie,Board Game,Singleplayer,..."
1509,SAMURAI WARRIORS 4-II,"Action,Hack and Slash,Historical,Character Cus..."
1510,Life is Hard,"Early Access,Early Access,Strategy,Indie,Simul..."
1511,Executive Assault,"Strategy,RTS,Action,FPS,Base Building,Robots,S..."


In [145]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tags_doc = [TaggedDocument(words=tokenization(tags), tags=[str(num)]) for num, tags in enumerate(df1.popular_tags.values)]
max_epochs = 50
alpha = 0.025

model = Doc2Vec(alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm=0)
  
model.build_vocab(tags_doc)
#training 
print('Epoch', end = ': ')
for epoch in range(max_epochs):
  print(epoch, end = ' ')
  model.train(tags_doc,
              total_examples=model.corpus_count,
              epochs=model.epochs)
  # decrease the learning rate
  model.alpha -= 0.0002
  # fix the learning rate, no decay
  model.min_alpha = model.alpha

Epoch: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

In [180]:
game_vectors = []
game_vectors2=[]
for game in df1["name"]:
    index = df1[df1["name"]==game].index.values[0]
    vec = model.dv[index]
    game_vectors.append(vec)
df1["vectors"] = game_vectors  
for i,game in enumerate(df["name"]):
    if game in df1["name"].values:
        index = df1[df1["name"]==game].index.values[0]
        vec = df1["vectors"][index]
        game_vectors2.append(vec)
    else: 
        game_vectors2.append("-")   
df["vectors"] = game_vectors2
df

,user_id,name,hours,desc_snippet,all_reviews,release_date,developer,publisher,popular_tags,game_details,...,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price,Rating M1,Rating M2,vectors
0,151603712,Fallout 4,87.0,"Bethesda Game Studios, the award-winning creat...","Mostly Positive,(90,387),- 70% of the 90,387 u...","Nov 9, 2015",Bethesda Game Studios,"Bethesda Softworks,Bethesda Softworks","Open World,Post-apocalyptic,Exploration,Single...","Single-player,Steam Achievements,Full controll...",...,RPG,"About This Game Bethesda Game Studios, the aw...",NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,$14.99,5,5,"[-0.122752674, 0.17151779, -0.1287731, 0.17929..."
1,87445402,Fallout 4,83.0,"Bethesda Game Studios, the award-winning creat...","Mostly Positive,(90,387),- 70% of the 90,387 u...","Nov 9, 2015",Bethesda Game Studios,"Bethesda Softworks,Bethesda Softworks","Open World,Post-apocalyptic,Exploration,Single...","Single-player,Steam Achievements,Full controll...",...,RPG,"About This Game Bethesda Game Studios, the aw...",NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,$14.99,5,5,"[-0.122752674, 0.17151779, -0.1287731, 0.17929..."
2,25096601,Fallout 4,1.6,"Bethesda Game Studios, the award-winning creat...","Mostly Positive,(90,387),- 70% of the 90,387 u...","Nov 9, 2015",Bethesda Game Studios,"Bethesda Softworks,Bethesda Softworks","Open World,Post-apocalyptic,Exploration,Single...","Single-player,Steam Achievements,Full controll...",...,RPG,"About This Game Bethesda Game Studios, the aw...",NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,$14.99,1,1,"[-0.122752674, 0.17151779, -0.1287731, 0.17929..."
3,211925330,Fallout 4,133.0,"Bethesda Game Studios, the award-winning creat...","Mostly Positive,(90,387),- 70% of the 90,387 u...","Nov 9, 2015",Bethesda Game Studios,"Bethesda Softworks,Bethesda Softworks","Open World,Post-apocalyptic,Exploration,Single...","Single-player,Steam Achievements,Full controll...",...,RPG,"About This Game Bethesda Game Studios, the aw...",NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,$14.99,3,5,"[-0.122752674, 0.17151779, -0.1287731, 0.17929..."
4,115396529,Fallout 4,17.9,"Bethesda Game Studios, the award-winning creat...","Mostly Positive,(90,387),- 70% of the 90,387 u...","Nov 9, 2015",Bethesda Game Studios,"Bethesda Softworks,Bethesda Softworks","Open World,Post-apocalyptic,Exploration,Single...","Single-player,Steam Achievements,Full controll...",...,RPG,"About This Game Bethesda Game Studios, the aw...",NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,$14.99,1,2,"[-0.122752674, 0.17151779, -0.1287731, 0.17929..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36252,154230723,Khet 2.0,1.9,Officially licensed version of the “Mensa Sele...,"Very Positive,(79),- 92% of the 79 user review...","Oct 1, 2014",BlueLine Games,"BlueLine Games,BlueLine Games","Strategy,Casual,Indie,Board Game,Singleplayer,...","Single-player,Multi-player,Cross-Platform Mult...",...,"Casual,Indie,Strategy",About This Game Officially licensed version o...,NaN,"Minimum:,OS:,Windows XP or better,Processor:,1...","Recommended:,OS:,Windows XP or better,Processo...",$9.99,NaN,1,5,"[-0.158661, -0.34231263, 0.4489203, -0.2071053..."
36253,116564064,SAMURAI WARRIORS 4-II,2.2,"The latest iteration of the 10th anniversary ""...","Mostly Positive,(860),- 77% of the 860 user re...","Sep 29, 2015","KOEI TECMO GAMES CO., LTD.","KOEI TECMO GAMES CO., LTD.,KOEI TECMO GAMES CO...","Action,Hack and Slash,Historical,Character Cus...","Single-player,Multi-player,Co-op,Steam Achieve...",...,Action,About This Game The

In [184]:
# example of how to recommend games based on a game
tags_vectors = model.docvecs
game = "MirrorMoon EP"
game_index = df1[df1["name"]==game].index.values[0]
#print(tags_vectors[game_index])
sims = model.docvecs.most_similar(positive = [game_index], topn = 10)

for i, j in sims:
  print(df1.loc[int(i), "name"].strip())

FRACT OSC
The Beginner's Guide
Kairo
Thirty Flights of Loving
Antichamber
Cubot
VVVVVV
Cosmonautica
hocus
FOTONICA


<ipython-input-184-b4fd690d008f>:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  tags_vectors = model.docvecs
<ipython-input-184-b4fd690d008f>:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar(positive = [game_index], topn = 10)


### Select a random user

### Rated games by the user

In [186]:
def get_rated_games_user(selected_user):
    selected_user_ratings = df.loc[df['user_id'] == selected_user]
    # selected_user_ratings = selected_user_ratings.sort_values(by='hours', ascending=True)
#     print("Rated games: " + str(selected_user_ratings.shape[0]))
    # display(selected_user_ratings.head(10))
    rated_games_df = selected_user_ratings

    rated_games_df = rated_games_df[['name', 'vectors', 'Rating M1', 'Rating M2']]

#     display(rated_games_df.head(10))    
    return rated_games_df


### Unrated games by the user

In [185]:
def get_unrated_games_user(rated_games_df):
    diff = set(df.index) - set(rated_games_df.index)
    unrated_games_df = df.loc[diff]
    unrated_games_df = unrated_games_df[['name', 'vectors']]
    unrated_games_df = unrated_games_df.drop_duplicates(subset ="name") #Had to remove duplicates 
#     print("Unrated games: " + .str(unrated_games_df.shape[0]))
#     display(unrated_games_df.head(10))
    return unrated_games_df

## Content Based Filtering
#### Remove users who dont have at least 30 games rated

In [90]:
import numpy as np
users = np.array(selected_users['user_id'])
unique, counts = np.unique(users, return_counts=True)
main_users =list()
for i, val in enumerate(unique):
    if counts[i] >= 30:
        main_users.append(val)

#### Train KNN for each user using Rating M1 and Rating M2 seperately for all users

In [212]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None  # default='warn'
# initialise the vectorizer
#vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
# vectorizer = CountVectorizer()
full_predictions = pd.DataFrame()
# for each gamer in the selected user list (min gameplay time of 300
for gamer in main_users:
    rated_games_df = get_rated_games_user(gamer)
#     display(rated_games_df)
    unrated_games_df = get_unrated_games_user(rated_games_df)
    X = rated_games_df["vectors"]
    y = rated_games_df['Rating M1']
    indices = np.arange(X.shape[0])
    (
        X_train,
        X_test,
        y_train,
        y_test,
        indices_train,
        indices_test,
    ) = train_test_split(X, y, indices, test_size=0.3, random_state=101)
    print(X_train.shape)
    neigh = KNeighborsRegressor(n_neighbors=10)
    neigh.fit(list(X_train), y_train)

    X_unrated = unrated_games_df['vectors']
    # X_unrated = X_unrated.reshape(1,-1)
    # print(X_unrated.shape)
    #predicting already existing rated values 
    y_pred1 = neigh.predict(list(X_test))
    #predicting unrated games
    y_pred1_unrated = neigh.predict(X_unrated)
#         print(y_pred1.shape)

    unrated_games_df['predicted_ratings_KNN M1'] = y_pred1_unrated
    tt = rated_games_df.iloc[indices_test]
    tt['y_test M1'] = y_pred1

    y2 = rated_games_df['Rating M2']
    indices2 = np.arange(X.shape[0])
    (
        X_train2,
        X_test2,
        y_train2,
        y_test2,
        indices_train2,
        indices_test2,
    ) = train_test_split(X, y2, indices2, test_size=0.3, random_state=101)
    neigh2 = KNeighborsRegressor(n_neighbors=10)
    neigh2.fit(list(X_train), y_train2)


    y_unrated2 = neigh2.predict(X_unrated)
    y_pred2 = neigh2.predict(list(X_test))

    unrated_games_df['predicted_ratings_KNN M2'] = y_unrated2
    unrated_games_df_KNN_ordering = unrated_games_df.sort_values(by='predicted_ratings_KNN M2', ascending=False)
#     display(unrated_games_df_KNN_ordering.head(10))

    tt['y_test M2'] = y_pred2
    full_predictions = full_predictions.append(tt)
    
#     tt = tt.sort_values(by='y_test M2', ascending=False)
#     display(tt.head(10))

(63,)


ValueError: setting an array element with a sequence.

In [188]:
display(full_predictions)

""


In [12]:
from sklearn.metrics import mean_squared_error
rmse_M1 = mean_squared_error(tt['Rating M1'], tt['y_test M1'])
rmse_M2 = mean_squared_error(tt['Rating M2'], tt['y_test M2'])
print('RMSE for M1: ' + str(rmse_M1))
print('RMSE for M2: ' + str(rmse_M2))

RMSE for M1: 2.3615000000000004
RMSE for M2: 2.854


## Collaborative Filtering 
#### UserUser with Lenskit

In [29]:
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser
ratings_df1 = df[['user_id','name','Rating M1']]
ratings_df1 = ratings_df1.rename(columns={'user_id':'user', 'name': 'item', 'Rating M1': 'rating'})
display(ratings_df1)
num_recs = 10  # Number of recommmendations to generate
user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(ratings_df1)

,user,item,rating
0,151603712,Fallout 4,5
1,87445402,Fallout 4,5
2,25096601,Fallout 4,1
3,211925330,Fallout 4,3
4,115396529,Fallout 4,1
...,...,...,...
36252,154230723,Khet 2.0,1
36253,116564064,SAMURAI WARRIORS 4-II,1
36254,135400225,Life is Hard,1
36255,135400225,Executive Assault,1


In [30]:
selected_games_useruser1 = recsys.recommend(selected_user, 10) # generate 10 recommendations for the selected user 

display(selected_games_useruser1)

,item,score
0,Fallout 4,4.677644
1,DeathSpank,4.656558
2,Fairy Fencer F,4.642703
3,EVE Online,4.621423
4,Impire,4.360516
5,Heroes of Scene,4.227792
6,Holy Potatoes! A Weapon Shop?!,4.187592
7,Kerbal Space Program,4.181591
8,Grim Dawn,4.161434
9,Elite Dangerous,4.072839


In [45]:
temp1 = unrated_games_df_KNN_ordering[['name','predicted_ratings_KNN M1']]
temp1 = temp1.rename(columns={'name':'item'})
temp1 = pd.merge(selected_games_useruser1, temp1, on ='item') 
temp1['Weighted Score'] = 0.5*temp1['score']  +  0.5*temp1['predicted_ratings_KNN M1']
final_pred1 = temp1[['item','Weighted Score']]
display(final_pred1)


,item,Weighted Score
0,Fallout 4,3.938822
1,DeathSpank,4.028279
2,Fairy Fencer F,3.621351
3,EVE Online,4.210711
4,Impire,3.080258
5,Heroes of Scene,3.913896
6,Holy Potatoes! A Weapon Shop?!,3.893796
7,Kerbal Space Program,3.790795
8,Grim Dawn,3.980717
9,Elite Dangerous,3.436420
